# Using a Web API

This notebook demonstrates using Python to access the OpenStreetMap RESTful API to get information about geographical locations. The code here is taken (with adaptions) from *Learning Python by Building Data Science Applications* by Philipp Kats and David Katz, chapter on geocoding with web APIs.

This code requires the `requests` package be installed. The `requests` package is part of the stock Anaconda installation, so if you are using Anaconda you don't need to do anything to install `requests`.

Let's make a basic request of the OpenStreetMap database using the Nominatim API:

In [31]:
import requests;

base_url = 'https://nominatim.openstreetmap.org/search?'
params = {
    "format" : "json",
    "q" : "201 Lawrence Pl., Williston, VT 05495"
}
result = requests.get(base_url, params)
print(result.status_code)

200


A status code of `200` means "success!" These are the standard HTTP status codes, so, for example, `404` means "resource not found." We asked for a result in JSON format, but we can ask the `result` object to convert that into a convenient Python data structure for us:

In [32]:
pythonic_result = result.json()
print(pythonic_result)

[{'place_id': 292867327, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 219862033, 'boundingbox': ['44.451261587211', '44.451361587211', '-73.1130796685', '-73.1129796685'], 'lat': '44.45131158721091', 'lon': '-73.1130296684996', 'display_name': '201, Lawrence Place, Taft Corners, Williston, Chittenden County, Vermont, 05495, United States', 'class': 'place', 'type': 'house', 'importance': 0.5309999999999999}]


What is returned by the `json` method is a list of dictionaries. It is a list because there might be multiple locations that match our query (in general). Each element of the list provides information about one location. If the query fails entirely, the returned list will be empty.

The dictionary that describes one of the search results provides a number of named attributes about the result. What interests us here is the longitude (the 'lon' attribute), and the latitude (the 'lat' attribute). Let's print a nice summary:

In [33]:
if len(pythonic_result) >= 1:
    location_information = pythonic_result[0]   # Ignore other results for now.
    print("For location:", params["q"])
    print("  Latitude  =", location_information["lat"])
    print("  Longitude =", location_information["lon"])

For location: 201 Lawrence Pl., Williston, VT 05495
  Latitude  = 44.45131158721091
  Longitude = -73.1130296684996


Let's define a function that prints the place information in a nice way.

In [34]:
def print_location(location_information):
    print("Location:", location_information["display_name"])
    print("  Latitude  =", location_information["lat"])
    print("  Longitude =", location_information["lon"])
    
print_location(pythonic_result[0])

Location: 201, Lawrence Place, Taft Corners, Williston, Chittenden County, Vermont, 05495, United States
  Latitude  = 44.45131158721091
  Longitude = -73.1130296684996


So far we have been working with the address of VTC's Williston campus. Now let's look up the GPS coordinates of VTC's main campus.

In [35]:
params = { "format" : "json", "q" : "124 Admin Dr., Randolph Center, VT 05061"}
result = requests.get(base_url, params)
if result.status_code != 200:
    print("Error! Request failed with status code:", result.status_code)
else:
    pythonic_result = result.json()
    if len(pythonic_result) == 0:
        print("Error! Request returned no matching locations")
    else:
        print_location(pythonic_result[0])

    

Location: Admin Drive, Randolph Center, Randolph, Orange County, Vermont, 05061, United States
  Latitude  = 43.9376948
  Longitude = -72.6012637


To compute the distance between two points on the Earth's surface given the GPS coordinates of those points, it is necessary to use spherical trigonometry. See the Wikipedia page on [great circle distance](https://en.wikipedia.org/wiki/Great-circle_distance) for more details. To illustrate the computation, lets start with the values returned above:

In [27]:
lat1 = 44.45131158721091
lon1 = -73.1130296684996

lat2 = 43.9376948
lon2 = -72.6012637

We need the absolute values of the differences. We also need to convert to radians:

In [28]:
import math

delta_lat = 2.0 * math.pi * (abs(lat1 - lat2)/360.0)
delta_lon = 2.0 * math.pi * (abs(lon1 - lon2)/360.0)

Now we use the haversine based formula from the web page:

In [36]:
t1 = math.sin(delta_lat/2.0)
t2 = t1*t1
t3 = math.sin(delta_lon/2.0)
t4 = t3*t3
t5 = t2 + math.cos(lat1) * math.cos(lat2) * t4
delta_sigma = 2.0 * math.asin(math.sqrt(t5))
print(delta_sigma)

0.012306678677552505


Knowing this angle (in radians) we can compute the distance on the surface of the Earth using the radius of the Earth:

In [37]:
radis_of_earth = (6378.137) * 0.62137119  # Convert kilometers to miles.
distance = radis_of_earth * delta_sigma
print("distance =", distance, "miles")

distance = 48.773712977325665 miles
